<a href="https://colab.research.google.com/github/rfqv/ProjectML_Python/blob/programPredCuaca/AplikasiPrediksiCuaca_LatML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [25]:
#!pip install --upgrade pyngrok

In [1]:
pip install requests

In [2]:
pip install googletrans==4.0.0-rc1

In [29]:
import requests
from googletrans import Translator
from datetime import datetime, timedelta

# Fungsi untuk mendapatkan City ID berdasarkan nama kota
def get_city_id(nama_kota, api_key='b99436d0ae93e7f8911a9e42f17c5ad2'):
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={nama_kota}&appid={api_key}"

    try:
        response = requests.get(base_url)
        data = response.json()

        if response.status_code == 200:
            id_kota = data['id']
            return id_kota
        else:
            return None  # Jika kota tidak ditemukan
    except Exception as e:
        print(f"Error: {e}")
        return None

# Fungsi untuk menerjemahkan deskripsi cuaca ke Bahasa Indonesia
def translate_description(description, dest_language='id'):
    translator = Translator()
    try:
        translated = translator.translate(description, dest=dest_language)
        return translated.text
    except Exception as e:
        print(f"Error during translation: {e}")
        return description  # Jika terjadi error, kembalikan deskripsi aslinya

# Fungsi untuk mengonversi waktu UTC ke waktu lokal berdasarkan timezone offset
def convert_utc_to_local(utc_time, timezone_offset):
    utc_dt = datetime.utcfromtimestamp(utc_time)  # Konversi dari Unix timestamp ke UTC datetime
    local_dt = utc_dt + timedelta(seconds=timezone_offset)  # Tambahkan offset zona waktu dalam detik
    return local_dt

# Fungsi untuk mendapatkan cuaca berdasarkan City ID
def get_weather_by_city_id(id_kota, api_key='b99436d0ae93e7f8911a9e42f17c5ad2'):
    base_url = f"http://api.openweathermap.org/data/2.5/forecast?id={id_kota}&appid={api_key}&units=metric"

    try:
        response = requests.get(base_url)
        data = response.json()

        if response.status_code == 200:
            nama_kota = data['city']['name']
            negara = data['city']['country']
            timezone_offset = data['city']['timezone']  # Mendapatkan offset zona waktu dalam detik
            weather_forecast = data['list'][0]  # Ambil forecast pertama (3 jam ke depan)

            temp = weather_forecast['main']['temp']
            feels_like = weather_forecast['main']['feels_like']
            humidity = weather_forecast['main']['humidity']
            wind_speed = weather_forecast['wind']['speed']
            wind_deg = weather_forecast['wind']['deg']
            visibility = weather_forecast.get('visibility', 'Tidak tersedia')
            pressure = weather_forecast['main']['pressure']
            weather_desc = weather_forecast['weather'][0]['description']

            weather_desc_id = translate_description(weather_desc, dest_language='id')

            # Mendapatkan waktu matahari terbit dan terbenam (dalam UTC)
            current_weather_url = f"http://api.openweathermap.org/data/2.5/weather?id={id_kota}&appid={api_key}&units=metric"
            current_weather_response = requests.get(current_weather_url)
            current_weather_data = current_weather_response.json()

            sunrise_utc = current_weather_data['sys']['sunrise']  # Matahari terbit dalam UTC
            sunset_utc = current_weather_data['sys']['sunset']  # Matahari terbenam dalam UTC

            # Konversi waktu matahari terbit dan terbenam ke waktu lokal
            sunrise_local = convert_utc_to_local(sunrise_utc, timezone_offset)
            sunset_local = convert_utc_to_local(sunset_utc, timezone_offset)

            # Menghitung GMT offset untuk menampilkan (dalam jam)
            gmt_offset = timezone_offset // 3600
            gmt_sign = "+" if gmt_offset >= 0 else "-"
            gmt_offset_str = f"GMT{gmt_sign}{abs(gmt_offset):02d}:00"

            result = f"""
            Laporan Prediksi Cuaca untuk Kota {nama_kota}, {negara} ({gmt_offset_str}):
            ---------------------------------------
            Suhu: {temp}\u00b0C
            Terasa seperti: {feels_like}\u00b0C
            Kelembaban: {humidity}%
            Tekanan Udara: {pressure} hPa
            Kecepatan Angin: {wind_speed} m/s
            Arah Angin: {wind_deg}\u00b0
            Visibilitas: {visibility} meter
            Matahari Terbit: {sunrise_local} (Waktu Lokal)
            Matahari Terbenam: {sunset_local} (Waktu Lokal)
            Kondisi: {weather_desc_id}
            """
            return result
        else:
            return f"Error: {data['message']} (Code: {data['cod']})"
    except Exception as e:
        return f"Error: {e}"

def show_weather_by_city_name():
    while True:
        nama_kota = input("Masukkan nama kota (atau ketik 'keluar' untuk keluar): ")
        if nama_kota.lower() == 'keluar':
            print("Program dihentikan. Terima kasih!")
            break

        if nama_kota:
            api_key = 'b99436d0ae93e7f8911a9e42f17c5ad2'
            id_kota = get_city_id(nama_kota, api_key)

            if id_kota:
                weather = get_weather_by_city_id(id_kota, api_key)
                print(weather)
            else:
                print("Kota tidak ditemukan! Harap periksa ejaan atau coba kota lain.")
        else:
            print("Harap masukkan nama kota!")

# Panggil kode ini untuk menjalankan program
show_weather_by_city_name()


Masukkan nama kota (atau ketik 'keluar' untuk keluar): Jakarta

            Laporan Prediksi Cuaca untuk Kota Jakarta, ID (GMT+07:00):
            ---------------------------------------
            Suhu: 30.97°C
            Terasa seperti: 35.03°C
            Kelembaban: 61%
            Tekanan Udara: 1005 hPa
            Kecepatan Angin: 5.77 m/s
            Arah Angin: 241°
            Visibilitas: 10000 meter
            Matahari Terbit: 2024-12-13 05:32:36 (Waktu Lokal)
            Matahari Terbenam: 2024-12-13 18:01:14 (Waktu Lokal)
            Kondisi: Awan yang rusak
            
Masukkan nama kota (atau ketik 'keluar' untuk keluar): keluar
Program dihentikan. Terima kasih!
